# Decorating Python 3

Following [this](https://python-3-patterns-idioms-test.readthedocs.io/en/latest/PythonDecorators.html) seems more reasonable than the official Python docs because those go directly to the `functools` library rather than actually explaining what decorators are and why they're important. And contextlib isn't mentioned at all.

In [35]:
class my_decorator(object):
    """A noisey, but simple decorator"""
    def __init__(self, func):
        """Called at '@' decoration time."""
        print('Inside my_decorator.__init__()')
        self.func = func
    
    def __call__(self, *args, **kwargs):
        """Called at func() run time."""
        print('Inside my_decorator.__call__()')
        print('About to run self.func...')
        self.func(*args, **kwargs)
        print('Done. Leaving my_decorator.__call__()')

@my_decorator
def my_func(*args, **kwargs):
    print('Inside my_func()')

Inside my_decorator.__init__()


I haven't even called `my_func()` yet and the decorator's `__init__` method gets run.

In [51]:
@my_decorator
def my_print_func_without_args():
    print('Inside my_print_func_without_args()')

@my_decorator
def my_print_func_with_args(*args, **kwargs):
    print('Inside my_print_func_with_args')
    for i, arg in enumerate(args):
        print('Arg in position {} is {}'.format(i, arg))
    for k, v in kwargs.items():
        print(' => '.join((k, v)))

print('About to call my_print_func_without_args...')
my_print_func_without_args()

print('About to call my_print_func_with_args...')
my_print_func_with_args('hello', 42, word='cats!')

Inside my_decorator.__init__()
Inside my_decorator.__init__()
About to call my_print_func_without_args...
Inside my_decorator.__call__()
About to run self.func...
Inside my_print_func_without_args()
Done. Leaving my_decorator.__call__()
About to call my_print_func_with_args...
Inside my_decorator.__call__()
About to run self.func...
Inside my_print_func_with_args
Arg in position 0 is hello
Arg in position 1 is 42
word => cats!
Done. Leaving my_decorator.__call__()


There's also a way to pass arguments to the decorators themselves without passing them to the function. Pass them to `__init__()` (if you're using a class), or just pass them to the decorator function if you're not using a class.

But passing arguments to the decorator function or class changes what your `__call__()` method or decorator function needs to return. It should return a wrapper function to the function being decorated, instead of the function being decorated directly.

In [26]:
# Note: copy pasta from the linked readthedocs.io above.
# PythonDecorators/decorator_with_arguments.py
class decorator_with_arguments(object):

    def __init__(self, arg1, arg2, arg3):
        """
        If there are decorator arguments, the function
        to be decorated is not passed to the constructor!
        """
        print("Inside __init__()")
        self.arg1 = arg1
        self.arg2 = arg2
        self.arg3 = arg3

    def __call__(self, f):
        """
        If there are decorator arguments, __call__() is only called
        once, as part of the decoration process! You can only give
        it a single argument, which is the function object.
        """
        print("Inside __call__()")
        def wrapped_f(*args):
            print("Inside wrapped_f()")
            print("Decorator arguments:", self.arg1, self.arg2, self.arg3)
            f(*args)
            print("After f(*args)")
        return wrapped_f

@decorator_with_arguments("hello", "world", 42)
def sayHello(a1, a2, a3, a4):
    print('sayHello arguments:', a1, a2, a3, a4)

print("After decoration")

print("Preparing to call sayHello()")
sayHello("say", "hello", "argument", "list")
print("after first sayHello() call")
sayHello("a", "different", "set of", "arguments")
print("after second sayHello() call")

Inside __init__()
Inside __call__()
After decoration
Preparing to call sayHello()
Inside wrapped_f()
Decorator arguments: hello world 42
sayHello arguments: say hello argument list
After f(*args)
after first sayHello() call
Inside wrapped_f()
Decorator arguments: hello world 42
sayHello arguments: a different set of arguments
After f(*args)
after second sayHello() call


### Everything in Context

Now I'm wondering if I can use `contextlib`'s **AbstractContextManager** as a base class for a custom decorator.

In [59]:
from contextlib import AbstractContextManager

class decore(AbstractContextManager):
    """Subclass of AbstractContextManager"""
    def __init__(self, func):
        """Decoration time"""
        print('AbstractContextManager::decore.__init__()')
        self.func = func
    
    def __enter__(self, *args, **kwargs):
        """Entry point of context manager"""
        print('AbstractContextManager::decore.__enter__()')
        self.func(*args, **kwargs)
        yield
    
    def __exit__(self, exc_type, exc_value, traceback):
        """Exit point of context manager
        
        According to the docs, the three arguments passed are the exception
        that was raised during either __enter__ or the body of the with
        statement (in our case, the function being decorated).
        
        If no exception was raised, all arguments will be None. 
        
        If the exception arguments are not None, as we wish to supress them,
        then we should return True here. Otherwise, they will propogate norm-
        ally after this method.
        """
        print('AbstractContextManager::decore.__enter__()')


#@decore
my_really_bad_math_function(*args, operation):
    print('my_function_with_args()')
    x, *y, z = *args
    if operation == 'add':
        return x + z
    elif operation == 'subtract':
        return x // y
    else:
        print('I can\'t do that operation, I\'ll guess though!')
        return 4



SyntaxError: invalid syntax (<ipython-input-59-2dedc9d01b6b>, line 33)

invalid syntax.. well, i may have to come back to this later..